In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 14.4MB/s 
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubun

In [2]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')
import re

Mounted at /content/drive


In [20]:
def to_str(num):
  if not (type(num) == str):
    num = round(num)
    num = str (num)
  return num

In [22]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

main_url = 'https://www.melon.com/song/detail.htm?songId='

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/일간멜론DJ/20210708일간멜론DJ.csv')


#nan값 지우기
df = df.fillna('')

#태그명 바꿔줌:
df = df.rename(columns={'노래_태그':'태그'})

genres = ['발라드', '댄스', '랩/힙합', 'R&B/Soul', '인디음악', '록/메탈', '트로트', '포크/블루스', 'POP', '록/메탈', '일렉트로니카', '랩/힙합', 'R&B/Soul', '포크/블루스/컨트리', 'OST', '클래식', '재즈', '뉴에이지', 'JPOP', '월드뮤직', 'CCM', '어린이/태교', '종교음악', '국악']

# 번호 전부 숫자로
df['번호'] = df['번호'].apply(to_str)

#에러가 나온 인덱스만 따로 모아놓자
err_list=[]

for idx in range(len(df['번호'].values)):
  url = main_url + df['번호'].iloc[idx]
  time.sleep(3)
  driver.get(url)

  try:
    body = driver.find_element_by_css_selector('body')
    res = driver.page_source
    soup = bs(res, "html.parser")

    #장르 업데이트
    print(df['장르'].iloc[idx], '->', end='')
    if not (df['장르'].iloc[idx] in genres):
      dl_list = soup.find('dl', {'class':'list'})
      dl_list_text = dl_list.text
      dl_list_text = dl_list_text.split('\n')
      genre_index = dl_list_text.index('장르')
      df['장르'].iloc[idx] = dl_list_text[genre_index+1]

    div_service_list_album = soup.find('div', {'class':'service_list_album'})
    tag_ul = div_service_list_album.find('ul')
    tag_div = tag_ul.find_all('div', {'class':'tag_list'})
    tag_list= []

    for div in tag_div:
      tag_list += div.text.replace('\n', ' ')[1:-1].split(' ')

    tag_list_ori = df['태그'].iloc[idx].split(' ')
    tag_list += tag_list_ori
    tag_set = set(tag_list)
    if '' in tag_set:
      tag_set.remove('')
    df['태그'].iloc[idx] = (' ').join(tag_set)
    
    print(df['장르'].iloc[idx], df['제목'].iloc[idx], df['태그'].iloc[idx], idx+1, '/', len(df['번호'].values))
  
  except:
    err_list.append(idx)
    continue

#파일로 저장
df.to_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/일간멜론DJ/20210708일간멜론DJ_전처리.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


 ->발라드 안녕, 여름밤 #발라드핫트랙 #인디 #여름밤 #사랑 #밤 #장르핫트랙 #신나는 #설렘 #새벽 #감성 1 / 2105


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


 ->인디음악, 포크/블루스 내(가) 너(를) #인디 #썸바디2 #사랑 #밤 #신나는 #설렘 #여름밤 #데이트 #감성 2 / 2105
 ->발라드, 인디음악 누가 봐도 연애 중 #한강 #인디 #사랑 #밤 #산책 #신나는 #설렘 #여름밤 #감성 3 / 2105
 ->인디음악, 록/메탈 속삭여줘 (DANG!) (Feat. 몽자) #취향저격 #Rock #인디 #사랑 #밤 #신나는 #설렘 #여름밤 #감성 4 / 2105
 ->인디음악, 록/메탈 Balloon #인디 #밤 #사랑 #신나는 #설렘 #기분전환 #힐링 #여름밤 #감성 5 / 2105
 ->인디음악, 포크/블루스 물감 (feat. 1ho) #트렌디 #인디 #이별 #사랑 #밤 #신나는 #설렘 #여름밤 #감성 6 / 2105
 ->R&B/Soul 로코는 로코다 (roco is roco) #피크닉 #인디 #사랑 #신나는 #집콕 #밤 #설렘 #여름밤 #감성 #카페 7 / 2105
 ->R&B/Soul, 인디음악 예쁜 말 (Art. 이민석) (Prod. GC) #취향저격 #트렌디 #인디 #사랑 #밤 #신나는 #설렘 #감성힙합 #여름밤 #감성 8 / 2105
 ->댄스, 인디음악 달라 #매장음악 #인디음악 #인디 #사랑 #신나는 #밤 #설렘 #여름밤 #감성 9 / 2105
 ->R&B/Soul, 인디음악 맥주 어때! #한강 #인디 #밤 #사랑 #산책 #신나는 #설렘 #여름밤 #감성 10 / 2105
 ->R&B/Soul, 인디음악 유영 #한강 #트렌디 #인디 #사랑 #케이팝 #산책 #밤 #신나는 #설렘 #여름밤 #감성 11 / 2105
 ->인디음악, 포크/블루스 알러지 #인디 #달달 #사랑 #밤 #신나는 #설렘 #여름밤 #감성 12 / 2105
 ->인디음악, 포크/블루스 너의 귓속말, 나의 거짓말 #인디 #사랑 #밤 #신나는 #슬픔 #설렘 #여름밤 #감성 #짝사랑 13 / 2105
 ->인디음악, 일렉트로니카 Dance with me baby #인디 #여름밤 #사랑 #밤 #신나는 #설렘 #시티팝 #감성 #레트로 14

KeyboardInterrupt: ignored

#테스트

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

main_url = 'https://www.melon.com/song/detail.htm?songId='

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/일간멜론DJ/20210708일간멜론DJ.csv')

#태그명 바꿔줌:
df = df.rename(columns={'노래_태그':'태그'})

genres = ['발라드', '댄스', '랩/힙합', 'R&B/Soul', '인디음악', '록/메탈', '트로트', '포크/블루스', 'POP', '록/메탈', '일렉트로니카', '랩/힙합', 'R&B/Soul', '포크/블루스/컨트리', 'OST', '클래식', '재즈', '뉴에이지', 'JPOP', '월드뮤직', 'CCM', '어린이/태교', '종교음악', '국악']

# 번호가 int형이라면 str로 바꿔줌
df['번호'] = df['번호'].apply(to_str)

idx = 0
url = main_url + df['번호'].iloc[idx]
time.sleep(3)
driver.get(url)

body = driver.find_element_by_css_selector('body')
res = driver.page_source
soup = bs(res, "html.parser")

#장르 업데이트
print(df['장르'].iloc[idx], '->', end='')
if not (df['장르'].iloc[idx] in genres):
  dl_list = soup.find('dl', {'class':'list'})
  dl_list_text = dl_list.text
  dl_list_text = dl_list_text.split('\n')
  genre_index = dl_list_text.index('장르')
  df['장르'].iloc[idx] = dl_list_text[genre_index+1]

div_service_list_album = soup.find('div', {'class':'service_list_album'})
tag_ul = div_service_list_album.find('ul')
tag_div = tag_ul.find_all('div', {'class':'tag_list'})
tag_list= []

for div in tag_div:
  tag_list += div.text.replace('\n', ' ')[1:-1].split(' ')

tag_list_ori = df['태그'].iloc[idx].split(' ')
tag_list += tag_list_ori
tag_set = set(tag_list)
if '' in tag_set:
  tag_set.remove('')
df['태그'].iloc[idx] = (' ').join(tag_set)

print(df['장르'].iloc[idx], df['제목'].iloc[idx], df['태그'].iloc[idx], idx+1, '/', len(df['번호'].values))

In [21]:
df.isna().sum()

Unnamed: 0       0
날짜               0
번호               0
제목               0
가수               0
장르            2089
앨범이름             0
좋아요              0
플레이리스트           0
태그               0
멜론DJ             0
플레이리스트_번호        0
dtype: int64